<a href="https://colab.research.google.com/github/ssvadla/Bridge_Project/blob/main/Supervised_Light_XG_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import time
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
train = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTrain.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.columns

Index(['Unnamed: 0', 'Skew', 'Number_Spans', 'Length_Max_Span',
       'Main_Design_Type', 'Deck_Design_Type', 'Wearing_Surface',
       'Deck_Protection', 'ADT', 'ADTT', 'State', 'Structure_Number',
       'Latitude', 'Longitude', 'Deck_Condition', 'Superstructure_Condition',
       'Substructure_Condition', 'Days_MinLT_0F', 'Days_MaxGT_90F',
       'Avg_Wind_Speed', 'Peak_Wind_Speed', 'Precipitation', 'Reconstructed',
       'Age', 'Deck_Area'],
      dtype='object')

In [4]:
#Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude', 'Superstructure_Condition', 'Substructure_Condition']
#Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude']
Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude','Days_MinLT_0F','Days_MaxGT_90F','Avg_Wind_Speed','Peak_Wind_Speed','Peak_Wind_Speed','Precipitation']

In [5]:
train = train.drop(columns = Excluded_columns)

In [6]:
train.columns

Index(['Skew', 'Number_Spans', 'Length_Max_Span', 'Main_Design_Type',
       'Deck_Design_Type', 'Wearing_Surface', 'Deck_Protection', 'ADT', 'ADTT',
       'Deck_Condition', 'Superstructure_Condition', 'Substructure_Condition',
       'Reconstructed', 'Age', 'Deck_Area'],
      dtype='object')

In [7]:
train.shape

(41902, 15)

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['Main_Design_Type_Encoded'] = le.fit_transform(train['Main_Design_Type'])
train['Deck_Design_Type_Encoded'] = le.fit_transform(train['Deck_Design_Type'])
train['Wearing_Surface_Encoded'] = le.fit_transform(train['Wearing_Surface'])
train['Deck_Protection_Encoded'] = le.fit_transform(train['Deck_Protection'])


In [9]:
 for value in train['Substructure_Condition']:
   #print(value)
   if value == 'N':
     print("Yes")

Yes


In [10]:
indexNames = train[ train['Substructure_Condition'] == 'N' ].index

In [11]:
indexNames

Int64Index([35702], dtype='int64')

In [12]:
train.shape

(41902, 19)

In [13]:
train.drop(indexNames , inplace=True)

In [14]:
train.shape

(41901, 19)

In [15]:
train['Substructure_Condition_typed']= pd.to_numeric(train['Substructure_Condition'])

In [16]:
np.unique(train['Substructure_Condition_typed'].values)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [17]:
train['Substructure_Condition_Encoded'] = le.fit_transform(train['Substructure_Condition_typed'])

In [18]:
import lightgbm as lgb
classifier_lgb = lgb.LGBMClassifier()

In [19]:
X = train.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection','Substructure_Condition_typed','Substructure_Condition']) 


In [20]:
y = train['Deck_Condition']

In [21]:
X.shape

(41901, 14)

In [22]:
X.columns

Index(['Skew', 'Number_Spans', 'Length_Max_Span', 'ADT', 'ADTT',
       'Superstructure_Condition', 'Reconstructed', 'Age', 'Deck_Area',
       'Main_Design_Type_Encoded', 'Deck_Design_Type_Encoded',
       'Wearing_Surface_Encoded', 'Deck_Protection_Encoded',
       'Substructure_Condition_Encoded'],
      dtype='object')

In [23]:
start = time.time()
classifier_lgb.fit(X, y)
end = time.time()
print(f"Runtime of the program is {end - start}")

Runtime of the program is 2.586982488632202


In [24]:
test = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTest.csv')

In [25]:
test = test.drop(columns = Excluded_columns)

In [26]:
test['Main_Design_Type_Encoded'] = le.fit_transform(test['Main_Design_Type'])
test['Deck_Design_Type_Encoded'] = le.fit_transform(test['Deck_Design_Type'])
test['Wearing_Surface_Encoded'] = le.fit_transform(test['Wearing_Surface'])
test['Deck_Protection_Encoded'] = le.fit_transform(test['Deck_Protection'])

In [27]:
#train['Substructure_Condition_typed']= pd.to_numeric(train['Substructure_Condition'])
test['Substructure_Condition_Encoded'] = le.fit_transform(test['Substructure_Condition'])

In [28]:
X_test = test.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection','Substructure_Condition']) 


In [29]:
y_test = test['Deck_Condition']

In [30]:
np.unique(train['Deck_Condition'].values)

array([ 6,  7, 45, 89])

In [31]:
np.unique(test['Deck_Condition'].values)

array([ 6,  7, 45, 89])

In [32]:
X_test.shape

(13970, 14)

In [33]:
test_pred = classifier_lgb.predict(X_test)

In [34]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test,test_pred, digits=4)

In [35]:
print(classification_report)

              precision    recall  f1-score   support

           6     0.6428    0.6081    0.6250      3861
           7     0.6754    0.7323    0.7027      4453
          45     0.7205    0.6985    0.7093      2746
          89     0.8161    0.7928    0.8043      2910

    accuracy                         0.7039     13970
   macro avg     0.7137    0.7079    0.7103     13970
weighted avg     0.7046    0.7039    0.7037     13970



In [36]:
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
classifier_xg = XGBClassifier()
start = time.time()
classifier_xg.fit(X, y)
end = time.time()
test_pred_xg = classifier_xg.predict(X_test)
classification_report_xg = classification_report(y_test,test_pred_xg, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_xg)

Runtime of the program is 9.667260646820068
              precision    recall  f1-score   support

           6     0.6436    0.5822    0.6114      3861
           7     0.6501    0.7355    0.6901      4453
          45     0.7153    0.6653    0.6894      2746
          89     0.8042    0.7973    0.8007      2910

    accuracy                         0.6922     13970
   macro avg     0.7033    0.6951    0.6979     13970
weighted avg     0.6932    0.6922    0.6913     13970



In [37]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
start = time.time()
classifier_rf.fit(X, y)
end = time.time()
test_pred_rf = classifier_rf.predict(X_test)
classification_report_rf = classification_report(y_test,test_pred_rf, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_rf)

Runtime of the program is 22.620065927505493
              precision    recall  f1-score   support

           6     0.6453    0.6211    0.6330      3861
           7     0.6844    0.7319    0.7073      4453
          45     0.7269    0.6988    0.7126      2746
          89     0.8187    0.8024    0.8105      2910

    accuracy                         0.7094     13970
   macro avg     0.7188    0.7135    0.7158     13970
weighted avg     0.7099    0.7094    0.7093     13970



In [38]:
from sklearn import svm
classifier_svm = svm.SVC(decision_function_shape='ovo')
start = time.time()
classifier_svm.fit(X, y)
end = time.time()
test_pred_svm = classifier_svm.predict(X_test)
classification_report_svm = classification_report(y_test,test_pred_svm, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_svm)

Runtime of the program is 133.33832931518555
              precision    recall  f1-score   support

           6     0.3740    0.0873    0.1415      3861
           7     0.3188    0.9347    0.4754      4453
          45     0.4615    0.0022    0.0043      2746
          89     0.0000    0.0000    0.0000      2910

    accuracy                         0.3225     13970
   macro avg     0.2886    0.2560    0.1553     13970
weighted avg     0.2957    0.3225    0.1915     13970



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
